In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("trongnguyen04/nckh-2425-crops")
print("Path to dataset files:", path)

# ─── copy raw folder “1” → /content/dataset_original/crops ───────
SRC="/root/.cache/kagglehub/datasets/trongnguyen04/nckh-2425-crops/versions/1"
DEST="/content/dataset_original/crops"
!mkdir -p $(dirname $DEST)
!cp -r "$SRC" "$DEST"

Path to dataset files: /kaggle/input/nckh-2425-crops
cp: cannot stat '/root/.cache/kagglehub/datasets/trongnguyen04/nckh-2425-crops/versions/1': No such file or directory


In [14]:
!git clone https://github.com/htrnguyen/compare_ocr_benchmark.git

Cloning into 'compare_ocr_benchmark'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 40 (delta 10), reused 38 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (40/40), 2.17 MiB | 18.24 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [6]:
!pip install python-Levenshtein jiwer ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 32.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [18]:
import sys, os, time
sys.path.append('/content/compare-ocr-benchmark/common')
sys.path.append('/content/compare_ocr_benchmark/CNN_TR_OCR_Resnet')

from image_preprocessing import preprocess_image
from metrics import compute_metrics
from utils import read_annotations, save_results

import pandas as pd
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import torch
from ultralytics import YOLO
from torchvision import transforms
import matplotlib.pyplot as plt

from model_cnn_transformer import OCRModel
from dataset_polygon import char2idx, idx2char

In [19]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)
VOCAB_SIZE = len(char2idx)
OCR_MODEL_PATH = "/content/models/best_ocr_model.pth"
YOLO_MODEL_PATH = "/content/models/best.pt"
FONT_PATH = "/content/compare_ocr_benchmark/CNN_TR_OCR_Resnet/Roboto-Regular.ttf"

yolo_model = YOLO(YOLO_MODEL_PATH)
ocr_model = OCRModel(vocab_size=VOCAB_SIZE).to(DEVICE)
ocr_model.load_state_dict(torch.load(OCR_MODEL_PATH, map_location=DEVICE))
ocr_model.eval()

def preprocess_ocr_image(pil_img):
    transform = transforms.Compose(
        [
            transforms.Resize((32, 128)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    )
    return transform(pil_img).unsqueeze(0)

def decode_sequence(indices):
    chars = []
    SOS_TOKEN = next((token for token in char2idx.keys() if "SOS" in token), None)
    for idx in indices:
        ch = idx2char.get(idx, "")
        if ch == "<EOS>":
            break
        if ch not in ("<PAD>", SOS_TOKEN):
            chars.append(ch)
    return "".join(chars)

Device: cpu


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 98.7MB/s]


RuntimeError: PytorchStreamReader failed reading zip archive: failed finding central directory

In [20]:
def group_text_lines(results, y_threshold=80, x_threshold=150):
    """
    Nhóm bbox thành dòng, trái-phải nếu 2 bbox gần nhau theo y.
    Trả về các dòng, mỗi dòng là list các từ theo thứ tự trái-phải, trên-xuống.
    """
    bboxes_with_text = []
    for bbox, text in results:
        x1, y1, x2, y2 = bbox
        y_center = (y1 + y2) / 2
        x_left = min(x1, x2)
        x_right = max(x1, x2)
        height = abs(y2 - y1)
        width = abs(x2 - x1)
        bboxes_with_text.append(
            {
                "bbox": bbox,
                "text": text,
                "y_center": y_center,
                "x_left": x_left,
                "x_right": x_right,
                "height": height,
                "width": width,
            }
        )

    # Sắp xếp ban đầu theo y_center (trên xuống dưới), x_left (trái-phải)
    bboxes_with_text.sort(key=lambda x: (x["y_center"], x["x_left"]))

    lines = []
    for item in bboxes_with_text:
        added = False
        for line in lines:
            if any(abs(item["y_center"] - i["y_center"]) < y_threshold for i in line):
                line.append(item)
                added = True
                break
        if not added:
            lines.append([item])
    for line in lines:
        line.sort(key=lambda x: x["x_left"])
    return lines

def join_grouped_lines_as_text(results, y_threshold=80):
    grouped_lines = group_text_lines(results, y_threshold=y_threshold)
    # Ghép text theo từng dòng, rồi lại ghép tất cả dòng thành 1 chuỗi, ngăn cách bởi dấu cách
    all_text = []
    for line in grouped_lines:
        text_line = " ".join([item["text"] for item in line])
        all_text.append(text_line)
    return " ".join(all_text).strip()

In [21]:
def yolo_cnntr_ocr_grouped(image_path, conf_threshold=0.5, y_threshold=80):
    # 1. Detection
    results = yolo_model(image_path)
    boxes = results[0].boxes.xyxy.cpu().numpy()
    confs = results[0].boxes.conf.cpu().numpy()
    filtered = [(box, conf) for box, conf in zip(boxes, confs) if conf > conf_threshold]
    boxes = [box for box, conf in filtered]

    # 2. Crop bbox và nhận diện text
    img_pil = Image.open(image_path).convert("RGB")
    results_list = []
    for box in boxes:
        x1, y1, x2, y2 = map(int, box)
        crop = img_pil.crop((x1, y1, x2, y2))
        image_tensor = preprocess_ocr_image(crop).to(DEVICE)
        with torch.no_grad():
            memory = ocr_model.encoder(image_tensor)
            SOS_TOKEN = next((token for token in char2idx.keys() if "SOS" in token), None)
            MAX_LEN = 36
            ys = torch.tensor([[char2idx[SOS_TOKEN]]], device=DEVICE)
            for _ in range(MAX_LEN):
                out = ocr_model.decoder(
                    ys,
                    memory,
                    tgt_mask=ocr_model.generate_square_subsequent_mask(ys.size(1)).to(DEVICE),
                )
                prob = out[:, -1, :]
                _, next_word = torch.max(prob, dim=1)
                ys = torch.cat([ys, next_word.unsqueeze(1)], dim=1)
                if next_word.item() == char2idx["<EOS>"]:
                    break
            pred_text = decode_sequence(ys.squeeze(0).tolist())
            results_list.append(((x1, y1, x2, y2), pred_text))
    # Gộp text đúng thứ tự dòng
    text_merged = join_grouped_lines_as_text(results_list, y_threshold=y_threshold)
    return text_merged

In [ ]:
CSV_ANN = '/content/dataset_original/crops/crops_gt.csv'
IMG_DIR = '/content/dataset_original/crops'

df = read_annotations(CSV_ANN)

In [ ]:
results = []

for idx, row in df.iterrows():
    fname = row['filename']
    desc_gt = row['description_gt']
    label = row.get('label', '')
    img_path = os.path.join(IMG_DIR, fname)

    try:
        t1 = time.perf_counter()
        pred = yolo_cnntr_ocr_grouped(img_path)
        t2 = time.perf_counter()
        infer_time = round(t2 - t1, 3)
    except Exception as e:
        pred = f"OCR_Error: {e}"
        infer_time = 0

    metrics = compute_metrics(desc_gt, pred)
    results.append({
        "filename": fname,
        "label": label,
        "ground_truth": desc_gt,
        "predicted_text": pred,
        "cer": metrics["cer"],
        "wer": metrics["wer"],
        "lev": metrics["lev"],
        "acc": metrics["acc"],
        "time": infer_time
    })

print("Done!")

In [ ]:
import os
OUT_CSV = '/content/compare-ocr-benchmark/results/cnntr_results.csv'
os.makedirs(os.path.dirname(OUT_CSV), exist_ok=True)
save_results(results, OUT_CSV)